# Lipid Micelle Self-assembly

This notebook prepares a DLPC micelle self assembly simulation.

This notebook is YELLOW because:

- Sage is not validated for lipids
- `pack_box` is an experimental private function

In [ ]:
from io import StringIO

import mdtraj
import nglview
import numpy as np
import openmm
from openff.interchange import Interchange
from openff.interchange.components._packmol import pack_box
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import unit

In [ ]:
dlpc = Molecule.from_file("inputs/dlpc.sdf")
lipids = [dlpc]

conc_nacl = 0.1 * unit.mole / unit.liter
n_waters = 8000
n_lipids = [25]
target_density = 1.0 * unit.gram / unit.milliliter

In [ ]:
water = Molecule.from_smiles("O")
na = Molecule.from_smiles("[Na+]")
cl = Molecule.from_smiles("[Cl-]")

for atom in water.atoms:
    atom.metadata["residue_name"] = "HOH"

for atom in na.atoms:
    atom.metadata["residue_name"] = "NA+"

for atom in cl.atoms:
    atom.metadata["residue_name"] = "CL-"

molarity_pure_water = 55.5 * unit.mole / unit.liter
n_nacl = int((n_waters / molarity_pure_water * conc_nacl).to(unit.dimensionless).m)
molecules = [*lipids, water, na, cl]
n_copies = [*n_lipids, n_waters, n_nacl, n_nacl]

total_mass = sum(
    [
        sum([atom.mass for atom in molecule.atoms]) * n
        for molecule, n in zip(molecules, n_copies)
    ]
)
target_volume = (total_mass / target_density).to(unit.nanometer**3)
box_vectors = np.asarray([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) * np.cbrt(target_volume)

top = pack_box(
    molecules,
    n_copies,
    box_vectors=box_vectors,
    tolerance=0.05 * unit.nanometer,
)

In [ ]:
top.to_file("micelle-self-assembly-start.pdb")

In [ ]:
w = top.visualize()
w.add_representation("line", selection="water")
w

In [ ]:
sage = ForceField("openff-2.2.0.offxml")

In [ ]:
interchange = Interchange.from_smirnoff(sage, top, charge_from_molecules=[dlpc])

In [ ]:
simulation = interchange.to_openmm_simulation(
    integrator=openmm.LangevinMiddleIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        2 * openmm.unit.femtosecond,
    )
)

In [ ]:
simulation.minimizeEnergy()
minimized_state = simulation.context.getState(
    getPositions=True, getEnergy=True, getForces=True
)

print(
    "Minimised to",
    minimized_state.getPotentialEnergy(),
    "with maximum force",
    max(
        np.sqrt(v.x * v.x + v.y * v.y + v.z * v.z) for v in minimized_state.getForces()
    ),
    minimized_state.getForces().unit.get_symbol(),
)

minimized_coords = minimized_state.getPositions()

In [ ]:
simulation.system.addForce(
    openmm.MonteCarloBarostat(
        1.0 * openmm.unit.bar,
        simulation.integrator.getTemperature(),
    )
)
simulation.context.reinitialize(preserveState=True)

In [ ]:
# Add a reporter to record the structure every data_freq steps
data_freq = 10000
dcd_reporter = openmm.app.XTCReporter("micelle-self-assembly.xtc", data_freq)
simulation.reporters.append(dcd_reporter)

state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    data_freq,
    step=True,
    potentialEnergy=True,
    kineticEnergy=True,
    temperature=True,
    density=True,
    elapsedTime=True,
    speed=True,
)
simulation.reporters.append(state_data_reporter)

In [ ]:
# simulation_time = 10 * unit.nanosecond
# steps = round((simulation_time / delta_t).to(unit.dimensionless).m)
# simulation.step(steps)

## Visualisation

In [ ]:
import mdtraj
import nglview

trajectory: mdtraj.Trajectory = mdtraj.load(
    # "micelle-self-assembly.xtc", 
    "micelle-self-assembly-backup.xtc", 
    # top=mdtraj.load("micelle-self-assembly-start.pdb").top, 
    top=mdtraj.load("micelle-self-assembly-start-backup.pdb").top, 
    stride=10,
)

# trajectory.image_molecules()

view = nglview.show_mdtraj(trajectory)
view.clear()
view.add_representation(
    "spacefill",
    selection=f"HOH NA+ CL-",
    opacity=0.1,
)
view.add_representation("licorice", selection="NOT (HOH NA+ CL-)")
view.add_unitcell()
view